In [4]:
import os
import cv2
import numpy as np
import imutils
import math

def main():
    if not os.path.exists('results'):
        os.makedirs('results')
    if not os.path.exists("BelgiumTSC_Training/"):
        os.makedirs("BelgiumTSC_Training/")
    
    
    
    directory = '00/'

    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f)
            
            if f.endswith('.ppm') or f.endswith('.jpeg') or f.endswith('.jp2'):
                    
                process(f)
def apply_mask(img,mask):
    
    imgsignalisation = img.copy()
    imgsignalisation[np.where(mask == 0)] = 0
    imgsignalisation[np.where(imgsignalisation != 0)] = 255
    
    
    # cv2.imshow('result', mask)
    # cv2.waitKey(0)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    contour_list = []
    best_ratio = 0
    maxLength = 0
    if len(contours) == 0:
        return -1
    for contour in contours:
        length = cv2.arcLength(contour,True)
        approx = cv2.approxPolyDP(contour,0.01*length, True)
        area = cv2.contourArea(contour)
        if area == 0:
            continue


        rayon = length/(2*math.pi)
        ratio = (math.pi*rayon**2) / (area)
        if ratio > 1:
            ratio = 1/ratio


        if ((len(approx) > 3) & (len(approx) < 20) ):
            if (area > maxLength*0.5 or ratio > best_ratio):
                maxLength = area
                
                
                best_ratio = ratio
                #del contour_list[:]
                contour_list.append(contour)
            elif (area == maxLength):
                contour_list.append(contour)
    if len(contour_list) == 0:
        return contours[0]

    # cv2.drawContours(image=imgoriginale,contours=contours,contourIdx=-1,color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
    # cv2.imshow('rr', imgoriginale)
    # cv2.waitKey(0)

    return contour_list

    

def contour_in_contour(contour1,contour2):
    
    for point in contour1:
        x = int(point[0][0])
        y = int(point[0][1])
        if cv2.pointPolygonTest(contour2,(x,y),False) < 0 : 
            return False
    return True

def process(file):
    
    # Import de l'image
    imgoriginale = cv2.imread(file, cv2.IMREAD_COLOR)
    # cv2.imshow("Originale", imgoriginale)

    # On resize si grande image
    height, width = imgoriginale.shape[:2]
    resized = imgoriginale.copy()
    
    lab = cv2.cvtColor(resized, cv2.COLOR_BGR2LAB)
    clahe = cv2.createCLAHE(clipLimit = 3., tileGridSize = (8, 8))
    l, a, b = cv2.split(lab)
    l2 = clahe.apply(l)
    lab = cv2.merge((l2, a, b))
    imgprocess = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
   
    # On floute avec du gaussian puis du median blur
    blur = cv2.GaussianBlur(imgprocess, (9, 9), 0)
    blur = cv2.medianBlur(blur, 9)

    # On transforme l'image en image HSV
    img_hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)



    # Définit la range du rouge en HSV
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])

    # Crée un masque pour la couleur rouge en utilisant la range définie
    mask1 = cv2.inRange(img_hsv, lower_red, upper_red)
    # Masque rouge valeurs hautes
    lower_red = np.array([170, 70, 50])
    upper_red = np.array([180, 255, 255])
    mask_red = cv2.inRange(img_hsv.copy(), lower_red, upper_red)

    mask_red = cv2.bitwise_or(mask1, mask_red)

    # Masque bleu
    lower_blue = np.array([90, 100, 100])
    upper_blue = np.array([121, 255, 255])
    mask_blue = cv2.inRange(img_hsv.copy(), lower_blue, upper_blue)

    # Jointure des masques pour les panneaux rouges et bleus
    
    lower_white = np.array([0, 0, 230])
    upper_white = np.array([200, 111, 255])
    mask_white = cv2.inRange(img_hsv.copy(), lower_white, upper_white)


    list_contours = []
    # Trouve les contours sur des images avec un filtre bleu, rouge et blanc
    list_blue = apply_mask(img_hsv,mask_blue)
    list_red = apply_mask(img_hsv,mask_red)
    list_white = apply_mask(img_hsv,mask_white,)

    if list_blue == -1 and list_red == -1:
        return 
    
    # Recherche d'un contour obtenu par un filtre d'une couleur totalement contenu dans un autre contour obtenu par un filtre d'une autre couleur
    # et vérifie que le rapport des aires des contours ne varie pas trop
    for blue in list_blue:
        for red in list_red:
            if contour_in_contour(blue,red):
                ratio = cv2.contourArea(blue)/cv2.contourArea(red)
                if ratio < 0.5 :
                    list_contours.append(red)
            elif contour_in_contour(red,blue):
                ratio = cv2.contourArea(red)/cv2.contourArea(blue)
                if ratio < 0.5 :
                    list_contours.append(blue)

    if list_blue == -1 and list_white == -1:
        return
    
    for white in list_white:
        for blue in list_blue:
            if contour_in_contour(white,blue):
                ratio = cv2.contourArea(white)/cv2.contourArea(blue)
                if ratio < 0.5 :
                    list_contours.append(blue)
            elif contour_in_contour(blue,white):
                ratio = cv2.contourArea(blue)/cv2.contourArea(white)
                if ratio < 0.5 :
                    list_contours.append(white)
    
    if list_white == -1 and list_red == -1:
        return
    
    for white in list_white:
        for red in list_red:
            if contour_in_contour(white,red):
                ratio = cv2.contourArea(white)/cv2.contourArea(red)
                if ratio < 0.5 :
                    list_contours.append(red)
            elif contour_in_contour(red,white):
                ratio = cv2.contourArea(red)/cv2.contourArea(white)
                if ratio < 0.5 :
                    list_contours.append(white)
    
    


    if len(list_contours) == 0: 
        print('ok')
        mask_tmp = cv2.bitwise_or(mask_white, mask_blue, mask= None)
        mask = cv2.bitwise_or(mask_blue, mask_red, mask= None)
        list_contours = apply_mask(img_hsv,mask)
    # Trie les contours par aire
    list_contours = sorted(list_contours, key=cv2.contourArea, reverse=True)
    
    # Trace un rectangle entourant le contour
    (x,y,w,h) = cv2.boundingRect(list_contours[0])
    
    cv2.rectangle(imgoriginale, (x,y), (x+w,y+h), (0,255,0), 2)
    cv2.drawContours(imgoriginale, list_contours,  -1, (255,0,0), 2)
    cv2.imshow('result', imgoriginale)
    cv2.waitKey()
    # raw_image = raw_image[y:y + h, x:x + w]

if __name__ == "__main__":
    main()

00/.directory
00/image.000058.jp2
00/image.000059.jp2
ok
00/image.000060.jp2
ok
00/image.000070.jp2
00/image.000071.jp2
ok
00/image.000072.jp2
ok
00/image.000089.jp2
00/image.000090.jp2
00/image.000091.jp2
00/image.000092.jp2
ok
00/image.000093.jp2
00/image.000094.jp2
00/image.000134.jp2
00/image.000163.jp2
00/image.000164.jp2
ok
00/image.000165.jp2
ok
00/image.000191.jp2
00/image.000192.jp2
00/image.000209.jp2


KeyboardInterrupt: 